In [165]:
import pandas as pd
import os,glob
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix



#path is that of the current directory
path = os.getcwd()
#print(location)

#empty list of corpus
corpus = []

#append each file with .txt extension to the corpus

for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        #print (filename)
        #print (len(text))
        corpus.append(text)

    
vectorizer = TfidfVectorizer(analyzer='word',stop_words = 'english', sublinear_tf=True)
tfidf_matrix = vectorizer.fit_transform(corpus)
#prints the tfidf MATRIX 
#print(tfidf_matrix)

#print(docs)


#feature_names = vectorizer.get_feature_names()
#corpus_index = [n for n in corpus]
#df = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
#print(df)

#remove stopwords 
#maxfeatures is to only get the top 5 relevant data
cv=CountVectorizer(analyzer='word', stop_words = 'english',lowercase=True,strip_accents='unicode')

# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(corpus)

word_count_vector.shape


tfidf_transformer=TfidfTransformer(smooth_idf=True)
tfidf_transformer.fit(word_count_vector)


count_vector=cv.transform(corpus)
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names()

#get tfidf vector for first document
#first_document_vector=tf_idf_vector[0]

#print the scores

#corpus_index = [n for n in corpus]

corpLength = len(corpus)

top5AllFiles = []


#build dataframe of first document. Determined by the index od tf-idf_vector below

corpusLength = len(corpus)

         
df = pd.DataFrame(tf_idf_vector[i].T.todense(), index=feature_names, columns= ["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

#get top 5 words
top5=df.nlargest(5, "tfidf")

array = []
data1=[]

for i, j in top5.iterrows():
               # print(i,j)
data1.append(i)
                data1.append(j.tfidf)
    
                array.append(data1)

                print(array[0])







IndentationError: unindent does not match any outer indentation level (<tokenize>, line 80)